# Studi Kasus 1 (Walkthrough)

<p> Pada kali ini Anda akan berperan sebagai seorang data engineer. Anda mendapatkan permintaan dari seorang *data analyst* dan *data scientist* untuk melakukan proses *data wrangling* pada sebuah dataset film untuk memastikan dataset tersebut siap dan layak untuk dianalisis oleh sang *data analyst* dan dapat menjadi basis pengembangan sebuah model prediktif yang sedang dikerjakan oleh sang *data scientist*</p>

<p>Dataset yang anda gunakan pada proyek ini berisi sejumlah daftar film beserta yang memuat informasi seperti:


*   Judul Film, Censorship Rating, Genre, Tahun Rilis, IMDb Score, Jumlah Vote IMDb,
*   Biaya Produksi (`budget`) dan Pemasukan Kotor (`gross`) yang dicetak masing-masing film
*   Flag yang merepresentasikan keterangan apakah film tersebut pernah dinominasikan/memenangkan kategori `*Best Picture*` pada *Academy Awards*.


Atas data tersebut, berikut detil informasi dan permintaan dari *data analyst*:
1.  Lakukan Penanganan Missing Values (Jika ada)
2.  Lakukan split atas kolom `released` sehingga didapatkan 2 kolom baru yakni `released_date` dan `released_country`. Lakukan penyesuaian format pada kolom `released_date` yang baru dibentuk sehingga memenuhi format **'yyyy-mm-dd'**
3.  Untuk setiap pemasukan kotor yang diperoleh masing-masing film, 50% di antaranya disetorkan ke Pihak Bioskop. Ini berarti Penghasilan yang diterima Pihak Studio hanya 50% dari total *gross*. Dengan informasi ini, hitunglah:
  * profit (50% gross - budget) yang dibukukan oleh masing-masing film.
  * Hitung pula profit yang telah disesuaikan berdasarkan tingkat inflasi pada tahun di mana masing-masing film tersebut dirilis!
4.   Kategorisasikan Profit (setelah disesuaikan dengan tingkat inflasi) ke dalam 5 kelas:
  * Rugi (Kurang dari 0)
  * 0 <= profit < 500,000
  * 500,000 <= profit < 1,000,000
  * 1,000,000 <= profit < 100,000,000
  * lebih dari 100,000,000

5.  Petakan kategori pada nomor 2 ke dalam list **[loss, low, medium, high, very high]**

Adapun permintaan dari *data scientist* adalah **lakukan transformasi atas seluruh *categorical attributes* yang ada ke dalam bentuk Encoding yang paling sesuai**




#0. Instalasi dan Import Library

In [ ]:
!pip install -U pandera

In [ ]:
!pip install missingno

In [ ]:
import pandas as pd
import numpy as np
import pandera as pa
import missingno as msno
from tqdm import tqdm
from datetime import datetime
import warnings, re

warnings.filterwarnings("ignore")

#1. Discovery and Preliminary Cleaning (Handling Missing Values)

In [ ]:
df = pd.read_csv(__)
display(df.head())

print("Jumlah Record: ", len(df))

In [ ]:
df.info()

##1.1 Check Missing Values

In [ ]:
df.isna().sum()

In [ ]:
msno.matrix(df)

In [ ]:
df  = df.sort_values(['year'], ascending=True)
df.head()

In [ ]:
df.describe()

In [ ]:
df[df.gross.isna()].describe()

In [ ]:
df.votes.describe()

In [ ]:
msno.heatmap(df)

Dari Hasil Identifikasi Missing Values didapatkan informasi sebagai berikut:

Kolom yang memiliki missing value

1. `rating`
<br />Tipe atribut (Categorical/Numerik): ****
<br />Tipe Missing Values (MCAR/MAR/MNAR): ****
<br />Strategi: ****
<br /><br />
2. `votes`
<br />Tipe atribut (Categorical/Numerik): ****
<br />Tipe Missing Values (MCAR/MAR/MNAR): ****
<br />Strategi: ****
<br /><br />
3. `country`
<br />Tipe atribut (Categorical/Numerik): ****
<br />Tipe Missing Values (MCAR/MAR/MNAR): ****
<br />Strategi: ****
<br /><br />
4. `gross`
<br />Tipe atribut (Categorical/Numerik): ****
<br />Tipe Missing Values (MCAR/MAR/MNAR): ****
<br />Strategi: ****
<br /><br />
5. `company`
<br />Tipe atribut (Categorical/Numerik): ****
<br />Tipe Missing Values (MCAR/MAR/MNAR): ****
<br />Strategi: ****
<br /><br />
6. `runtime`
<br />Tipe atribut (Categorical/Numerik): ****
<br />Tipe Missing Values (MCAR/MAR/MNAR): ****
<br />Strategi: ****
<br /><br />
5. `best_pic_award`
<br />Tipe atribut (Categorical/Numerik): ****
<br />Tipe Missing Values (MCAR/MAR/MNAR): ****
<br />Strategi: ****
<br /><br />

##1.2 Handling Missing Values

###1.2.1 Rating

In [ ]:
# untuk atribut rating dengan missing values, kita akan mencari 2 genre tertinggi
df[df.rating.isna()]['genre'].value_counts().head(2).index

In [ ]:
# temukan rating tertinggi untuk kedua genre tersebut
df[df.genre.isin(['Drama','Horror'])]['rating'].value_counts()

In [ ]:
# lakukan imputation dengan men-set nilai missing values pada kolom rating dengan rating tertinggi yang ditemukan sebelumnya
df.rating.fillna(__, inplace=True)

In [ ]:
df.isna().sum()

###1.2.2 Votes

In [ ]:
# load data movies_votes yang digunakan pada votes enrichment
df_votes = pd.read_csv(__)
df_votes.columns = ["votes_"+col for col in list(df_votes.columns )]
df_votes.head()

In [ ]:
# lakukan join antara df dengan df_votes dengan menggunakan 3 atribut sebagai join key: judul film, tahun rilis, dan imdb score
df_merge = pd.merge(df, df_votes, how=__, left_on=[__], right_on=[__])

In [ ]:
checked_rows = list(df_merge[df_merge.votes.isna()].index)
checked rows

In [ ]:
votes_from_enrichment = df_merge.loc[df_merge.votes.isna(),'votes_votes']
votes_na = df_merge.votes.isna(), 'votes'

# proses imputation dengan menggunakan vote yang didapatkan dari prose enrichment
df_merge.loc[votes_na] = __
df_merge.loc[checked_rows]

In [ ]:
# drop seluruh kolom hasil enrichment pasca imputasi
df = df_merge[list(df.columns)]
display(df.head())
print("Number of records: ", len(df))
df.isna().sum()

###1.2.3. Country

In [ ]:
df[df['country'].isna()]

In [ ]:
#lakukan deletion pada kolom country
df.dropna(subset=[__], inplace=True)
df.isna().sum()

###1.2.4 Runtime

In [ ]:
df[df['runtime'].isna()]

In [ ]:
# sedikit googling akan memberikan informasi runtime atas film tersebut yakni: 91 menit
df['runtime'].fillna(91, inplace=True)
df.isna().sum()

###1.2.5 Company

In [ ]:
df[df.company.isna()]

In [ ]:
#lakukan proses deletion pada atribut company
df.dropna(subset=[__], inplace=True)
df.isna().sum()

###1.2.6 Gross

In [ ]:
# berikut adalah daftar genre dari setiap record dengan missing values pada atribut gross
missing_genres = list(df[df['gross'].isna()].groupby(['genre'])['name'].count().index)
missing_genres

In [ ]:
# Asumsi: seluruh missing values akan di-impute dengan menggunakan median untuk masing-masing genre pada 1981
for genre in missing_genres:
  rows_to_impute = (df['genre'] == __) & (df['gross'].isna())
  median = df[(df['genre'] == __) & (df['year'] == __)].median()['gross']

  checked_rows = list(df.loc[rows_to_impute, :].index)

  print(f"genre: {genre}")
  print(f"median: {median}")

  df.loc[rows_to_impute, 'gross'] = median
  display(df.loc[checked_rows, 'gross'])

In [ ]:
df.isna().sum()

In [ ]:
# setelah diteliti ternyata masih terdapat 1 missing value pada atribut gross. Hal ini disebabkan pada proses imputasi sebelumnya,
# record tersebut tidak berhasil diupdate karena pada tahun 1981 tidak terdapat record dengan genre Fantasy.
df[df['genre'] == 'Fantasy'].year.value_counts().sort_index()

In [ ]:
# karena jumlah film pada genre Fantasy terbatas, maka proses imputasi akan menggunakan median dari gross film Fantasy pada rentang waktu 1981-1989
condition = (df['genre'] == 'Fantasy') & (df['year'].isin(list(range(__,__))))
fantasy_median = df[condition]['gross'].median()

In [ ]:
df.loc[(df['genre'] == 'Fantasy') & (df['gross'].isna()), 'gross'] = __
df.isna().sum()

###1.2.7 Best Picture Award

In [ ]:
df.best_pic_award.value_counts()

In [ ]:
df.best_pic_award.isna().sum()

In [ ]:
# lakukan "missing imputation" untuk atribut best_pic_award
df.best_pic_award.fillna(__, inplace=True)
df.best_pic_award.value_counts()

In [ ]:
df.isna().sum()

#2. Further Cleaning and Enrichment

##2.1 Split kolom `released` menjadi kolom `released_date` dan kolom `released_country`

In [ ]:
def clean_date(d, format="%Y-%m-%d"):
  try:
    d = re.findall(__, d)[0]
    return datetime.strftime(datetime.strptime(d, "%B %d, %Y"), format)
  except:
    return d

def get_released_country(d):
  try:
    return re.findall(__, string)[0]
  except:
    return d

# sanity check
string =  "December 18, 1989 (United States)"
print(clean_date(string))
print(get_released_country(string))


1989-12-18
United States


In [ ]:
# lakukan cleansing dengan menggunakan kedua fungsi di atas berdasarkan nilai dari kolom `released`
df['released_date'] = __
df['released_country'] = __
df.head()

##2.2. Hitung Profit

###2.2.1 Hitung *Unadjusted Gross* (50% dari gross yang belum disesuaikan dengan inflasi)

In [ ]:
df['unadjusted_gross'] = __
df[['gross','unadjusted_gross']].head()

###2.2.2 Hitung *Unadjusted Profit* (`unadjusted_gross` - `budget`, belum disesuaikan dengan inflasi)

In [ ]:
df['unadjusted_profit'] = __
df[['name','year','budget','gross','unadjusted_gross','unadjusted_profit']].sample(5)

###2.2.3 Hitung Adjusted Profit menggunakan Data Inflasi (*enrichment*)

In [ ]:
df_inflation = pd.read_csv("__")
df_inflation

,year,present_equivalent_value
0,1956,11.32
1,1957,10.96
2,1958,10.66
3,1959,10.56
4,1960,10.41
...,...,...
62,2018,1.23
63,2019,1.20
64,2020,1.19
65,2021,1.14


In [ ]:
#proses enrichment dengan menggunakan df_inflasion berdasarkan tahun rilis sebagai join key
df = pd.merge(__)
df.head()

In [ ]:
# hitung adjusted profit dengan menggunakan present_equivalent_value
df['adjusted_profit'] = __
df[['name','year','budget','gross','unadjusted_gross','unadjusted_profit', 'present_equivalent_value','adjusted_profit']].sample(5)

###2.2.4 Kategorisasi Profit

In [ ]:
df['adjusted_profit'].describe()

In [ ]:
# lakukan kategorisasi adjusted profit berdasarkan bracket/bins yang tertera pada deskripsi di awal project ini.
# labels=['loss', 'low_profit', 'medium_profit', 'high_profit', 'very_high_profit']

df['profit_class'] = pd.cut(__,
       bins=[__],
       labels = [__],
       include_lowest=True)
df[['name','adjusted_profit','profit_class']]

In [ ]:
df.head()

In [ ]:
df[df['profit_class'].isna()]

In [ ]:
selected_cols = ['name','rating','genre','year','released_date','score',
                 'director','budget','unadjusted_gross','unadjusted_profit','adjusted_profit','profit_class']
df = df[selected_cols]
df.head()

#3. Verifying

In [ ]:
#verifikasi data menggunakan library pandera

schema = pa.DataFrameSchema(
    {
        "year": pa.Column(int, pa.Check.isin(list(range(1980,2023)))),
        "budget": pa.Column(int, pa.Check.greater_than(0))
    }
)
schema.validate(df)

#4. Publishing

In [ ]:
df_for_analyst = df.copy()
df_for_datsci = df.copy()

print("Number of Records for Dat Analyst: ", len(df_for_analyst))
print("Number of Records for Data Scientist: ", len(df_for_datsci))

Number of Records for Dat Analyst:  5433
Number of Records for Data Scientist:  5433


##4.1 For Data Analyst

In [ ]:
df_for_analyst.head()

korelasi antara imdb score dengan adjusted profit. Apakah tingginya *tingginya imdb score* berkorelasi dengan nilai *adjusted profit*?

In [ ]:
# hitung korelasi antara imdb score dan adjusted profit

Kelompokkan data berdasarkan *profit_class* dan bandingkan jumlah film yang dikategorikan ke dalam masing-masing kategori pada *profit_class*. Buatlah sebuah visualisasi atas perbandingan tersebut!

In [ ]:
df_for_analyst.groupby(__)['name'].__().plot.bar(rot=45)

Ekspor data final ke dalam sebuah file **movies_clean.csv**

In [ ]:
df_for_analyst.__

##4.2 For Data Scientist

Metode encoding apa yang tepat untuk *profit_class*? One-Hot Encoding atau Label Encoding?

In [ ]:
# kategori profit class
df_for_datsci[__].dtype.categories

In [ ]:
profit2class = {c:i for i,c in enumerate(__.to_list())}
profit2class

In [ ]:
df_for_datsci['profit_class'] = df_for_datsci['profit_class'].apply(lambda x: profit2class.get(x))
df_for_datsci.head()

Metode encoding apa yang tepat untuk *rating*? One-Hot Encoding atau Label Encoding?

In [ ]:
rating_hierarchy = ['Approved','G','PG','PG-13','R','NC-17','X','TV-MA','Not Rated','Unrated']
rating2class = {r:i for i, r in enumerate(rating_hierarchy)}
rating2class

In [ ]:
df_for_datsci['rating'] = df_for_datsci['rating'].apply(lambda x: rating2class.get(x))
df_for_datsci.head()

Metode encoding apa yang tepat untuk *genre*? One-Hot Encoding atau Label Encoding?

In [ ]:
df_for_datsci = df_for_datsci.join(pd.get_dummies(df['genre'], 'genre'))
df_for_datsci.head()

In [ ]:
columns  = list(df_for_datsci.columns)

In [ ]:
genre_cols = [c for c in columns if c.startswith("genre_")]

In [ ]:
new_cols = columns[:2] + genre_cols + columns[3:12]
df_for_datsci = df_for_datsci[new_cols]
df_for_datsci.head()

Ekspor data final untuk data scientist ke sebuah **movies_clean_for_datsci.csv** file